#### Simulation

##### Import

In [7]:
import numpy as np
import itertools
import requests
import polyline
import json
import os
import random as rd
import pandas as pd

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from shapely.geometry import Point

import warnings 

warnings.filterwarnings('ignore')

##### 좌표 거리 생성 함수

In [36]:
def calculate_straight_distance(lat1, lon1, lat2, lon2):

    km_constant = 3959* 1.609344
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    km = km_constant * c
    
    return km

##### trips 데이터 생성 함수

In [34]:
# library
#############
# OSRM base #
#############
def get_res(point):

   status = 'defined'

   session = requests.Session()
   retry = Retry(connect=3, backoff_factor=0.5)
   adapter = HTTPAdapter(max_retries=retry)
   session.mount('http://', adapter)
   session.mount('https://', adapter)

   overview = '?overview=full'
   loc = f"{point[0]},{point[1]};{point[2]},{point[3]}" # lon, lat, lon, lat
   # url = "http://127.0.0.1:5000/route/v1/driving/"
   url = 'http://router.project-osrm.org/route/v1/driving/'
   r = session.get(url + loc + overview) 
   
   if r.status_code!= 200:
      
      status = 'undefined'
      
      # distance    
      distance = calculate_straight_distance(point[1], point[0], point[3], point[2]) * 1000
      
      # route
      route = [[point[0], point[1]], [point[2], point[3]]]

      # duration & timestamp
      speed_km = 10#km
      speed = (speed_km * 1000/60)      
      duration = distance/speed
      
      timestamp = [0, duration]

      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
   
      return result, status
   
   res = r.json()   
   return res, status

##############################
# Extract duration, distance #
##############################
def extract_duration_distance(res):
       
   duration = res['routes'][0]['duration']/60
   distance = res['routes'][0]['distance']
   
   return duration, distance

#################
# Extract route #
#################
def extract_route(res):
    
    route = polyline.decode(res['routes'][0]['geometry'])
    route = list(map(lambda data: [data[1],data[0]] ,route))
    
    return route

#####################
# Extract timestamp #
#####################
def extract_timestamp(route, duration):
    
    rt = np.array(route)
    rt = np.hstack([rt[:-1,:], rt[1:,:]])

    per = calculate_straight_distance(rt[:,1], rt[:,0], rt[:,3], rt[:,2])
    per = per / np.sum(per)

    timestamp = per * duration
    timestamp = np.hstack([np.array([0]),timestamp])
    timestamp = list(itertools.accumulate(timestamp)) 
    
    return timestamp
 
########
# MAIN #
########

# - input : O_point, D_point (shapely.geometry.Point type)
# - output : trip, timestamp, duration, distance
def osrm_routing_machine(O, D):
       
   osrm_base, status = get_res([O.x, O.y, D.x, D.y])
   
   if status == 'defined':
      duration, distance = extract_duration_distance(osrm_base)
      route = extract_route(osrm_base)
      timestamp = extract_timestamp(route, duration)

      result = {'route': route, 'timestamp': timestamp, 'duration': duration, 'distance' : distance}
      
      return result
   else: 
      return osrm_base

def osrm_routing_machine_multiprocess(OD):
   O, D = OD
   result = osrm_routing_machine(O, D)
   return result

def osrm_routing_machine_multiprocess_all(OD_data):
    results = list(map(osrm_routing_machine_multiprocess, OD_data))
    return results

##### od 데이터 생성

In [26]:
# 랜덤한 쌍의 od 데이터(시작점과 도착점이 같이 않게 뜸)
def get_OD_data(point, num = 10) :
    random_pairs = []

    for _ in range(num):
        neighborhood1, neighborhood2 = rd.sample(point.keys(), 2)
        start_point = point[neighborhood1]
        end_point = point[neighborhood2]
        random_pairs.append([start_point, end_point])

    OD_data = []
    
    for i, (start_point, end_point) in enumerate(random_pairs, 1):
        OD_data.append({
            "O_name": list(point.keys())[list(point.values()).index(start_point)],
            "O": Point(start_point),
            "D_name": list(point.keys() )[list(point.values()).index(end_point)],
            "D": Point(end_point)
        })
        
    OD_data = pd.DataFrame(OD_data)
    
    return OD_data

In [17]:
# 데이터 좌표
point = {
    "가천대역" : [127.126858, 37.449621],
    "가천대_반도체대학" : [127.127384 , 37.450910],
    "가천대_일반대학원" : [127.130112 , 37.452589],
    "가천대_교육대학원" : [127.131698 , 37.452066],
    "가천대_학생회관" : [127.134042 , 37.453336],
    "가천대_ai_공학관" : [127.133374 , 37.455009],
}

# 가천대역 = Point(127.126858, 37.449621)
# 가천대_반도체대학 = Point(127.127384 , 37.450910)
# 가천대_일반대학원 = Point(127.130112 , 37.452589)
# 가천대_교육대학원 = Point(127.131698 , 37.452066)
# 가천대_학생회관 = Point(127.134042 , 37.453336)
# 가천대_ai_공학관 = Point(127.133374 , 37.455009)

In [28]:
OD_data = get_OD_data(point, 10)
OD_data

,O_name,O,D_name,D
0,가천대_반도체대학,POINT (127.127384 37.45091),가천대_일반대학원,POINT (127.130112 37.452589)
1,가천대_일반대학원,POINT (127.130112 37.452589),가천대역,POINT (127.126858 37.449621)
2,가천대_ai_공학관,POINT (127.133374 37.455009),가천대_일반대학원,POINT (127.130112 37.452589)
3,가천대_일반대학원,POINT (127.130112 37.452589),가천대역,POINT (127.126858 37.449621)
4,가천대_ai_공학관,POINT (127.133374 37.455009),가천대_학생회관,POINT (127.134042 37.453336)
5,가천대역,POINT (127.126858 37.449621),가천대_일반대학원,POINT (127.130112 37.452589)
6,가천대_ai_공학관,POINT (127.133374 37.455009),가천대_학생회관,POINT (127.134042 37.453336)
7,가천대역,POINT (127.126858 37.449621),가천대_교육대학원,POINT (127.131698 37.452066)
8,가천대_학생회관,POINT (127.134042 37.453336),가천대_교육대학원,POINT (127.131698 37.452066)
9,가천대역,POINT (127.126858 37.449621),가천대_ai_공학관,POINT (127.133374 37.455009)


In [29]:
## od 데이터를 이용하기 위해 리스트쌍으로 변환
def get_ps_OD_data(OD_data):
    OD_point = []
    DO_point = []
    for index, row in OD_data.iterrows():
        O = Point(row['O'])
        D = Point(row['D'])
        OD_point.append([O, D])
        DO_point.append([D, O])

    return OD_point, DO_point

In [30]:
OD_point, DO_point = get_ps_OD_data(OD_data)

##### trips 데이터 생성

In [83]:
OD_results = osrm_routing_machine_multiprocess_all(OD_point)
DO_results = osrm_routing_machine_multiprocess_all(DO_point)

##### timestamp 변경

In [79]:
# 갔다가 돌아오는 경로이기 때문에 타임스탬프가 이루어져야 한다
def timestamp_change(OD_results, DO_results) :
    for i in range(0, len(OD_results)) :
        DO_results[i]['timestamp'] = list(DO_results[i]['timestamp'] + OD_results[i]['timestamp'][-1] + (0+1)*10)
        OD_results[i]['timestamp'] = list(np.array(OD_results[i]['timestamp']) + (0+1)*10)
    return OD_results, DO_results

In [84]:
OD_results, DO_results = timestamp_change(OD_results, DO_results)

In [89]:
trips = OD_results + DO_results

In [92]:
path = '../simulation/src/data/'

with open(os.path.join(path + 'trips.json'), 'w', encoding='utf-8') as file:
    json.dump(trips, file)